## Imports

In [1]:
import os, sys
import re
import itertools as it
import numpy as np

""" 
If you have problem importing, you probably don't have BioPython.
    Try calling 'pip install biopython' in your terminal
"""
from Bio import SeqIO
from Bio import Data
from Bio import Seq
from Bio import SeqRecord

## Set-up + Data Downloading

In [2]:
""" 
The folder that should contain all data,
    Include protein databases (references), and
    DNA sequences (querys)
"""
data_directory = "data"

if not os.path.exists(data_directory):
    os.makedirs(data_directory)

**PLEASE** go to data folder and call "bash get_data.sh"

## 6-Frame Translation

In [3]:
def read_query_file(file_name, file_format='fasta'):
    """
    Parse a fasta or fastq file into pairs of name and sequences.
    
    Args:
            file_name (str): Name of the file containing query sequences
            file_format (str): Must be either 'fasta' or 'fastq'.
            
    Returns:
            query_dict (dict): A dictionary object containing all parsed queries
    """
    
    # Check for file format
    if file_format.lower() not in ['fasta', 'fastq']:
        raise Exception("Not recognized file format")
    
    # An iterator of parsed object
    query_iter = SeqIO.parse(file_name, file_format.lower())
    
    # Turn the iterator into a dictionary 
    if file_format == 'fasta':
        query_dict = SeqIO.to_dict(query_iter)
    else:
        """
        If there is duplicate keys, append an
            incrementing value to the reads' ID
        """
        alternative = {}
        for idx,i in enumerate(query_iter):
            alternative[f"{i.id}_{idx}"] = i
        return alternative
    return query_dict

In [4]:
def translate_seq(sequence):
    """
    Translate a BioPython BioSeq object that contains
    DNA sequences.
    
    Args:
            sequence (Bio.Seq): A Bio.Seq object that contains DNA->Amino Acid translation function
            
    Returns:
            cds (list): A list containing all valid CDS with a minimum length of 20
    """
    min_cds_length = 10
    
    # Translate amino acid past any stop codon
    aa  = sequence.translate()
    
    # Extract all valid CDS (starts with M and ends with stop codon)
    regr= re.compile(r'M[A-Z]*')
    cds = regr.findall(str(aa))
    
    # filter out short cds
    cds = [c for c in cds if len(c) > min_cds_length]
    
    return cds

In [5]:
def six_frame_translation(sequence):
    """
    Wrapper function the calls translate_seq at all 6 frames
    
    Args:
            sequence (Bio.Seq): A Bio.Seq object that contains DNA->Amino Acid translation function
            
    Returns:
            cds (list): A list containing all 6-frames' CDSs
    """
    # Obtain the reverse complement strand
    rc_sequence = sequence.reverse_complement()
    
    # Aggregate CDS from all 6 frames
    six_frame = [translate_seq(sequence), 
                 translate_seq(sequence[1:]), 
                 translate_seq(sequence[2:]), 
                 translate_seq(rc_sequence), 
                 translate_seq(rc_sequence[1:]),
                 translate_seq(rc_sequence[2:])]
    
    # flatten the list
    six_frame = [cds for cds_list in six_frame for cds in cds_list]

    return six_frame

# Reverse-Translation

In [6]:
print(Data.CodonTable.standard_dna_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

In [7]:
reverse_codon = {'A': ['GCT', 'GCC', 'GCA', 'GCG'], 
                 'C': ['TGT', 'TGC'],
                 'D': ['GAT', 'GAC'], 
                 'E': ['GAA', 'GAG'],
                 'F': ['TTT', 'TTC'], 
                 'G': ['GGT', 'GGC', 'GGA', 'GGG'], 
                 'H': ['CAT', 'CAC'],
                 'I': ['ATT', 'ATC', 'ATA'], 
                 'K': ['AAA', 'AAG'],
                 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], 
                 'M': ['ATG'], 
                 'N': ['AAT', 'AAC'], 
                 'P': ['CCT', 'CCC', 'CCA', 'CCG'], 
                 'Q': ['CAA', 'CAG'], 
                 'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 
                 'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], 
                 'T': ['ACT', 'ACC', 'ACA', 'ACG'],
                 'V': ['GTT', 'GTC', 'GTA', 'GTG'], 
                 'W': ['TGG'], 
                 'Y': ['TAT', 'TAC']}

To help Bowtie2 or any aligner that uses Burrow Wheeler Transform (BWT), we would like to find a set of reverse translation mapping that yields the highest overall entropy. This highest overall entropy is computed by summing across per-base-position entropy. In another word, we want the distribution of A,C,G,T for each individual base position (namely, 1st, 2nd, and 3rd base) to be as diverse as possible. Doing so can hopefully allow us to maximally reduce the number of rows that we are searching, each time we perform FM indexing. And we choose to use entropy to measure the diversity of A,C,G,T.

In [8]:
def compute_entropy(letters):
    """
    Given a list of letters, compute the entropy for letter distributions
    
    Args:
            letters (list): a list of consists of A,C,G,T 
            
    Returns:
            entropy (float): a value representing information entropy.
                             $ - \sum \frac{1}{p} \log_2 \frac{1}{p}$
    """
    # Compute frequency
    A = letters.count('A') / len(letters)
    C = letters.count('C') / len(letters)
    G = letters.count('G') / len(letters)
    T = letters.count('T') / len(letters)
    
    # Pseudo-count
    counts = np.array([A, C, G, T]) + 1e-5
    
    # compute entropy
    probas = counts / np.sum(counts)
    ents   = probas * np.log2(probas)
    entropy= -1 * np.sum(ents)
    return entropy

#### Brute Force

**Don't** try to run the following code. It is **NOT** intended to be executed.

The following code should be able to return the reverse codon mapping with the highest entropy; however, because it tries to brute force all possible combinations of mapping, it will take too much time. 

#### Heuristic

A summary table that aggregates all possible mapping for each individual amino acid. The columns are "Abbreviated amino acid", "1st base", "2nd base", "All Possible 3rd bases", "Alternative 2nd base", and "Corresponding alternative 3rd bases".

We can see that W(Tryptophan) and M(Methionine) have only 1 possible reverse mapping.

15 amino acids have fixed 1st-base and 2nd-base with 2~4 choises over their 3rd-base.

3 amino acids have fixed 1st-base but 2 choices of 2nd-base and corresponding 3rd-bases.

Upon observing the summary of codon table (shown above), we see that most variances lie in the 3rd position of codons. Because the majority of amino acids have fixed first two positions of codons already, we can try to come up with a heuristic reverse-mapping by hand.

First of all, we see that all the amino acids except S(Serine) has a choice of C or G on the 2nd base. Given that the 2nd-base distribution for all other 19 amino acids is [A,C,G,T]:[7,3,4,5], we prefer C over G, for they yield entropy of 1.9589 and 1.9406, respectively. 

We will take a greedy approach and claim that the second position is now fixed.

Given that we have choosen C as the second base for S(Serine), we have only 2 choices left to make to finalize the first base position. L(Leucine) can have a C or a T for its first base, and R(Arginine) can have either a C or a A for its first base. The 1st-base distribution for all other 18 amino acids is [A,C,G,T]: [5,3,5,5]. Because we know the uniform distribution yields the highest entropy, we will have both Leucine and Arginine to take C as its first base.

Now that we have both the 1st-base position and 2nd-base position resolved. We only need to figure out what to do for the 3rd-base position. Because there are so much variances for this base position, there exists multiple combinations that lead to uniform distribution. One of such mapping is
E/Q/K/W/M takes G
I/F/C/D/H tales T 
Y/N/P/T/V takes C
A/G/L/R/S takes A.

While we took a greedy approach in selecting the reverse-mapping, we argue that we indeed arrived at the set of mapping with the highest overall entropy. Because we have uniform distribution over A,C,G,T for both the 1st-base position and the 3rd-base position, there is no room to improve upon it. Meanwhile, we have gotten the highest entropy possible for the 2nd-base position, thus, we claim that we found our ideal mapping.

In [9]:
ideal_r_mapping = {'E': 'GAG', 'Q': 'CAG', 'K': 'AAG', 'I': 'ATT', 
                   'F': 'TTT', 'C': 'TGT', 'D': 'GAT', 'H': 'CAT', 
                   'Y': 'TAC', 'N': 'AAC', 'P': 'CCC', 'T': 'ACC', 
                   'V': 'GTC', 'A': 'GCA', 'G': 'GGA', 'L': 'CTA', 
                   'R': 'CGA', 'S': 'TCA', 'W': 'TGG', 'M': 'ATG'}

# Encoding

In [10]:
def encode_sequences(fname, out_name, reference=True, file_type='fasta'):
    """
    This function handles reverse mapping from amino acid to nucleotide
    
    Args:
            fname (str): the relative path to the input file
            out_name (str): the relative path to where the output should be stored
            reference (bool): where this is a reference file or a query file. If latter,
                              the reads will first get translated into protein and then
                              reverse-mapped back to nucleotide space.
            file_type (str): whether the file is a fasta or a fastq.
            
    Returns:
            None -- output is directly written to the output file
            
    """
    # Read querys into SeqRecord objects
    entries = read_query_file(fname, file_format=file_type)
    print(f"Processed {fname:>80} with {len(entries):>30} records")    

    # If a protein reference, directly map to nt space
    if reference:
        for k,v in entries.items():
            encoded = [ideal_r_mapping[letter]  if letter in ideal_r_mapping else 'CGT' for letter in v.seq]
            v.seq   = Seq.Seq(''.join(encoded)) 
          
    # If a nucleotide query, first do 6-frame translation and then map back to nt space
    else:      
        # Populate a new list with transformed CDSs
        new_entries = {}
        counter     = 0 
        
        # For each query
        for record, (k, v) in enumerate(entries.items()):
            six_frame = six_frame_translation(v.seq)

            # for each valid CDS region
            for idx, cds in enumerate(six_frame):
                encoded = [ideal_r_mapping[letter] if letter in ideal_r_mapping else 'CGT' for letter in cds]
                new_seq = Seq.Seq(''.join(encoded))
                new_entries[counter] = SeqRecord.SeqRecord(seq=new_seq, id=f"{v.id}_{record}&*#{idx}", name=f"{v.id}_{record}&*#{idx}", description="")
                counter += 1
                
        entries = new_entries
        
    
    # Output encoded sequences
    with open(out_name, "w") as handle:
        SeqIO.write(entries.values(), handle, "fasta")
        
    return a

In [11]:
"""
Main function:

1. Process all the protein references. Map records into nucleotide space.

2. Process all the query sequences. Obtain possible (valid and long enough) 
        CDS from each reads, and then map each CDS individual back into nucleotide
        space.
        
TAKES ABOUT 5 MINUTES ON MY LAPTOP
"""
lol=0
# List all files in the data directory
data_files = os.listdir(data_directory)
for file in data_files:
    in_name  = os.path.join(data_directory,file)
    
    # Find all the protein references
    if '_protein.faa' in file:
        out_name = in_name.replace('_protein.faa', '_aa_encoded.fasta')
        lol=encode_sequences(in_name, out_name, True, 'fasta')
        
    # Find all raw sequencing reads
    elif 'fastq' in file:gi
        out_name = in_name.replace('.fastq', '_nt_encoded.fasta')
        lol=encode_sequences(in_name, out_name, False, 'fastq')

Processed                          data\GCF_000820495.2_ViralMultiSegProj14656_protein.faa with                             10 records
Processed                                  data\GCF_000864765.1_ViralProj15476_protein.faa with                             10 records
Processed                                  data\GCF_000882815.3_ViralProj36615_protein.faa with                              1 records
Processed                                 data\GCF_000901155.1_ViralProj183710_protein.faa with                             11 records
Processed                         data\GCF_001343785.1_ViralMultiSegProj274766_protein.faa with                             11 records
Processed                                     data\GCF_002816195.1_ASM281619v1_protein.faa with                             11 records
Processed                                     data\GCF_009858895.2_ASM985889v3_protein.faa with                             12 records
Processed                                              

C:\Users\garyc\Anaconda3\envs\tf2\lib\site-packages\Bio\Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
